In [ ]:
!pip install openpyxl

In [ ]:
from bs4 import BeautifulSoup
import urllib.error,os
import urllib.request
import time,datetime
import openpyxl
import pandas as pd
import numpy as np
import glob
import pprint
import re

In [ ]:
import_file_path = '\価格調査.xlsx'
excel_sheet_name = 'ブース'#'ブース', 'テー']
ccc = 3 #入力位置
xxx = 2 #スタート
export_file_path = '.output'

In [ ]:
df_order = pd.read_excel(import_file_path, sheet_name = excel_sheet_name)
wb = openpyxl.load_workbook('\価格調査.xlsx')
sheet = wb[excel_sheet_name]
Pcell = sheet.cell(row=ccc, column=1)
dt_now = datetime.datetime.now()
Pcell.value = dt_now.strftime('%Y/%m/%d')
wb.save('\価格調査.xlsx')
print(dt_now.strftime('%Y/%m/%d'))

In [ ]:
ua = 'K'
num = 2
while True:
    cell = sheet.cell(row=5, column=num)
    #print(cell.value)
    if cell.value==None:
        break
    num += 1
print(num)
for i in range(xxx, num):#2
    time.sleep(2)
    Ucell = sheet.cell(row=7, column=i)
    Rcell = sheet.cell(row=5, column=i)
    req = urllib.request.Request(Ucell.value, headers = {'User-Agent': ua})
    html = urllib.request.urlopen(req)
    soup = BeautifulSoup(html, "html.parser")
    c = 'error'  
    xList = ['D']
    if Rcell.value in xList :
        c = 'xed'    
    elif Rcell.value == 'P':
        c = 'P' 
    elif Rcell.value == 'SP':
        c = 'Special'        
    elif Rcell.value == 'S':
        c = 'SR'
    else:
        print(c)
    print(i)

    print(Ucell.value) 
    #print(Rcell.value)#レアリティを表示
    print(c)
    pre = soup.find_all(class_="A")#ありそうなところを検索
    for z in pre:
        if 'class="B"' in str(z) and "C" not in str(z) and c in str(z):
            seed = str(z)
            target1 = 'href="'
            target2 = '"><span'
            idx1 = seed.find(target1)
            idx2 = seed.find(target2)
            r =  "https://www.XXX.com"+ seed[idx1+len(target1):idx2]
            print(r)#URLを表示
            req = urllib.request.Request(r, headers = {'User-Agent': ua})
            html = urllib.request.urlopen(req)
            soup = BeautifulSoup(html, "html.parser")        
    Rare_list = soup.find_all(class_="D")
    p = 10000.0
    for v in Rare_list:
        if c == 'error':
            break
        #print(v)#コードを表示
        #print()
        target1 = c
        idx = str(v).find(target1)
        y = str(v)[idx+len(target1):]
        if target1 == "E":
            for j in range(4):
                idx = y.find(target1)
                y = y[idx+len(target1):]
        #print(y)
        print()
        target2 = 'F'
        idx = y.find(target2)
        print(idx)
        if idx > 0:
            y = y[:idx]
        #print(y)
        Price_list = v.find_all(itemprop="G")
        for w in Price_list:
            #print(w)
            #print(y)
            if str(w) in y or c == "H":
                seed = str(w)
                target1 = 'content="'
                idx1 = seed.find(target1)
                seed = seed[idx1+len(target1):]
                target2 = '" I"J"'
                idx2 = seed.find(target2)
                seed = seed[:idx2]
                print(seed)
                if float(p) > float(seed) or p == 0:
                    p = float(seed)
                    #print(p)
    Pcell = sheet.cell(row=ccc, column=i)
    if p == 0:
        Pcell.value = 'soldout'
    else:
        Pcell.value = p
    wb.save('\価格調査.xlsx')        
#soup